In [1]:
from unityagents import UnityEnvironment
import numpy as np

from collections import deque
import torch
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
from IPython import display

from ddpg_agent import Agents



# env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")
env = UnityEnvironment(file_name="Tennis.app", worker_id=1000)

brain_name = env.brain_names[0]
brain = env.brains[brain_name]

env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]

print('Number of agents:', num_agents)
print('Size of each action:', action_size)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))

agent = Agents(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=0)


def ddpg(n_episodes=1000, max_t=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations
        agent.reset()
        score = np.zeros(num_agents)
        for t in range(max_t):
            action = agent.act(state)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
                        
            agent.step(state, action, rewards, next_state, dones)
            state = next_state
            score += rewards
            
            
            if np.any(dones):
                break 
        scores_deque.append(np.mean(score))
        scores.append(np.mean(score))
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_tennis.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_tennis.pth')   
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

env.close()

INFO:matplotlib.font_manager:font search path ['/Users/nickvazz/miniconda3/envs/drlnd/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf', '/Users/nickvazz/miniconda3/envs/drlnd/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/afm', '/Users/nickvazz/miniconda3/envs/drlnd/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
INFO:matplotlib.font_manager:Could not open font file /Library/Fonts/NISC18030.ttf
INFO:matplotlib.font_manager:generated new fontManager
INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space si

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24


TypeError: __init__() missing 9 required positional arguments: 'fc1_units', 'fc2_units', 'BUFFER_SIZE', 'BATCH_SIZE', 'GAMMA', 'TAU', 'LR_ACTOR', 'LR_CRITIC', and 'CRITIC_WEIGHT_DECAY'